In [1]:
import bson 
import numpy as np
import pandas as pd
import time
import sys
import scipy.sparse
import heapq

In [2]:
with open("linkedin_db/profile.bson",'rb') as f: 
    data_profile = bson.decode_all(f.read()) 
    f.close()

In [3]:
delete_word = ['de','-','et','&','des','en','du','and','/','of','la','au','in',
                   'à','for','the',':','at','|','le','–','on','to','dans','entreprise',
                   '»', 'En', 'as', 'un','les', 'et','pour','avec']

In [4]:
dict_tran_word = {
    'gestion':'management',
    'stratégie': 'strategy',
    'd\'équipe' : 'team',
    'développement': 'development',
    'commerciale' : 'commercial',
    'réseaux': 'network',
    'réseau':'network',
    'networking':'network',
    'networks'
    'stratégique' : 'strategic',
    'sociaux' : 'social',
    'français':'french',
    'négociations': 'negotiation',
    'négociation':'negotiation',
    'travail':'work',
    'humaines': 'human',
    'droit':'law',
    'financière': 'finance',
    'mobiles':'mobile',
    'vente':'selling',
    'ventes':'selling',
    'publiques': 'public',
    'ingénierie': 'engineering',
    'systèmes':'system',
    'systems':'system',
    'sécurité':'security',
    'd\'énergie':'energy',
    'énergie':'energy',
    'énergétique':'energy',
    'd\'évènement': 'event',
    'd\'évènements':'event',
    'média':'media',
    'médias':'media',
    'd\'entreprise':'enterprise',
    'juridique':'legal',
    'rédaction':'writting',
    'amélioration':'improvement',
    'télécommunications':'telecommunications',
    'logistique':'logistics',
    'intégration':'integration',
    'numérique':'digital',
    'comptabilité':'accounting',
    'financier':'financial',
    'banque':'banking',
    'agiles':'agile',
    'espagnol':'spanish',
    'politique':'policy',
    'politiques':'policy',
    'entrepreneuriat':'entrepreneurship',
    'achats': 'purchasing',
    'modélisation':'modeling',
    'internationales':'internationale',
    'contrôle':'control',
    'controls':'control',
    'technologies':'technology',
    'technique':'technical',
    'contentieux':'litigation',
    'mécanique': 'mechanical',
    'analyses':'analyse',
    'qualité':'quality', 
    'utilisateur':'user',
    'd\'entreprise': 'entreprise',
    'électronique': 'electronics',
    'électrique': 'electrical',
    'vidéo':'video',
    'markets':'market',
    'marché':'market',
    'expérience':'experience',
    'mathématiques':'mathematics',
    'avant-vente': 'pre-sales',
    'databases':'database',
    'renouvelable':'renewable',
    'projet':'project',
    'recrutement':'recruitment',
    'chaîne':'chain',
    'pétrole':'oil',
    'nucléaire':'nuclear',
    'produits':'produit',
    'apprentissage':'learning',
    'embarqués':'embedded',
    'crédit':'credit',
    'compétences':'skills',
    'd\'investissement':'investment',
    'aéronautique':'aviation',
    'opérationnelle':'operational',
    'opérations':'operation',
    'globale': 'global',
    'dérivés': 'derivatives',
    'prévisions':'forecasting',
    'journalisme':'journalism',
    'histoire':'history',
    'aménagement':'planning',
    'aeronautics':'aerospace',
    'création':'creation',
    'étude':'study'
}

In [5]:
skill_list = []
start = time.clock()
for i in range(len(data_profile)):
    if 'skills' in data_profile[i].keys():
        for skills in data_profile[i]['skills']:
            if 'skills' in skills.keys():
                for skill in skills['skills']:
                    for item in skill['title'].split():
                        if item.lower() in delete_word:
                            break
                        if item.lower() in dict_tran_word:
                            skill_list.append(dict_tran_word.get(item.lower()))
                        else:
                            skill_list.append(item.lower())
print(time.clock()-start)

1.29750818568548


In [6]:
from collections import Counter
num_feature = 500
word_counts = Counter(skill_list)
top_n = word_counts.most_common(num_feature)
print(top_n)

[('management', 79853), ('microsoft', 25839), ('marketing', 24408), ('strategy', 23652), ('business', 12024), ('project', 11598), ('office', 11274), ('development', 11059), ('team', 10364), ('commercial', 8467), ('social', 7388), ('communication', 7358), ('excel', 7357), ('negotiation', 6630), ('engineering', 6379), ('change', 6262), ('french', 6059), ('anglais', 5722), ('network', 5674), ('relations', 5547), ('digital', 5247), ('service', 5189), ('finance', 5142), ('powerpoint', 5104), ('leadership', 5019), ('word', 4621), ('analysis', 4479), ('data', 4388), ('java', 4351), ('consulting', 4183), ('public', 4177), ('web', 3995), ('planification', 3971), ('sql', 3970), ('media', 3784), ('planning', 3730), ('client', 3672), ('law', 3607), ('financial', 3482), ('communications', 3334), ('it', 3334), ('analyse', 3294), ('human', 3292), ('design', 3222), ('sales', 3045), ('adobe', 3042), ('english', 3026), ('energy', 3015), ('selling', 2911), ('international', 2713), ('security', 2708), ('f

In [7]:
feature = []
proportion = []
for i in top_n:
    feature.append(i[0])
    proportion.append(i[1])
coff = 1./np.sqrt(proportion)

In [8]:
df = pd.DataFrame(data=[],index=[],columns=feature)
start = time.clock()
array = scipy.sparse.lil_matrix((len(data_profile), len(feature)))
top_effect = 10
for i in range(len(data_profile)):
    rang = np.zeros(len(feature))
    effect = 1
    if 'skills' in data_profile[i].keys():
        for skills in data_profile[i]['skills']:
            if 'skills' in skills.keys():
                if skills['title'] == 'Top Skills':
                    effect = top_effect
                else:
                    effect = 1
                for skill in skills['skills']:
                    for item in skill['title'].split():
                        if item.lower() in dict_tran_word:
                            item = dict_tran_word.get(item.lower())
                        if item.lower() in feature:
                            index = feature.index(item.lower())
                            array[i,index] = max(array[i,index],1 * effect)
df = pd.DataFrame(data=array.A, columns=feature)
print(time.clock()-start)

11.435852196228897


In [9]:
def match(df, index, n_output):
    loss = 0.01
    sample = coff * df.loc[index].values
    if sum(sample) == 0:
        print('This profile doesn\'t have skill.')
    sample = sample - loss*(sample==0)
    score = df.values.dot(sample)
    score = list(score)
    max_index = heapq.nlargest(n_output, range(len(score)), score.__getitem__)
    return max_index

In [10]:
def show_skill(index):
    show = [[],[]]
    show[0].append('Top Skills: ')
    show[1].append('Other Skills: ')
    for skills in data_profile[index]['skills']:
        if skills['title'] == 'Top Skills':
            for skill in skills['skills']:
                show[0].append(skill['title'])
        else:
            for skill in skills['skills']:
                show[1].append(skill['title'])
    print(show[0])
    print(show[1])
    return

In [11]:
random_index = np.random.choice(len(data_profile), 10, replace=False)
random_index

array([29101,  7012, 28872,  6728,  4626, 23714, 49146, 26374, 45840,
       22683])

# example on the no.31929 profile

In [12]:
match(df, 31929, 3)

[31929, 30380, 2979]

In [13]:
show_skill(31929)

['Top Skills: ', 'Science actuarielle', 'Statistiques', 'R']
['Other Skills: ', 'SAS', 'Visual Basic pour Applications (VBA)', 'Microsoft Access', 'Microsoft Office']


In [14]:
show_skill(30380)

['Top Skills: ', 'Statistiques', 'R', 'SAS']
['Other Skills: ', 'VBA', 'Microsoft Excel', 'Microsoft Access']


In [15]:
show_skill(2979)

['Top Skills: ', 'SAS', 'Statistiques', 'R']
['Other Skills: ', 'Finance', 'Gestion des risques', 'C++', 'Java']


# example on the no.11103 profile

In [16]:
match(df, 11103, 10)

[11103, 5067, 4865, 5022, 5035, 4632, 9972, 11133, 4581, 9776]

In [17]:
show_skill(11103)

['Top Skills: ', 'Énergie', 'Microsoft Excel', 'Nuclear Engineering']
['Other Skills: ', 'Java', 'Python', 'Nuclear Physics', 'MCNP', 'Open Office', 'DIF3D']


In [18]:
show_skill(5067)

['Top Skills: ', 'Ingénierie', 'Sûreté nucléaire', 'Énergie']
['Other Skills: ', 'Management', 'démantèlement']


In [19]:
show_skill(4865)

['Top Skills: ', 'Ingénierie', 'Énergie', 'Nucléaire']
['Other Skills: ', 'Satisfaction du client', 'Réactivité']


# example on the no.18436 profile

In [20]:
match(df, 18436, 3)

[7219, 18436, 38146]

In [21]:
show_skill(18436)

['Top Skills: ', 'Microsoft Office', 'Management', 'Project Management']
['Other Skills: ', 'Adobe Photoshop', 'Mac', 'Gestion de projet', "Management d'équipe"]


In [22]:
show_skill(7219)

['Top Skills: ', 'Adobe Photoshop', 'Gestion de projet', 'Microsoft Office']
['Other Skills: ', 'Project Management', 'Marketing', 'Marketing Communications', 'Business Development', 'Photoshop', 'HTML', 'Microsoft Excel', 'Microsoft Word', 'Adobe Illustrator', 'CSS', 'Management', 'English', 'French', 'Français', 'Développement commercial', 'Stratégie commerciale', 'Communication marketing', "Communication d'entreprise", 'Communication visuelle', 'comunity management', 'Anglais', 'Réseaux sociaux']


In [23]:
show_skill(39146)

['Top Skills: ', 'Management', 'Team Management', 'Microsoft Office']
['Other Skills: ', 'Microsoft Excel', 'Microsoft Word', 'Coaching', 'Ressources humaines', "Gestion d'équipe", 'Gestion des talents', 'Gestion des performances', 'Anglais']


# try to use clustering method

In [24]:
from sklearn.cluster import KMeans
start = time.clock()
kmeans = KMeans(n_clusters=100, random_state=0).fit(df)
print(time.clock() - start)

140.38147664709268


In [25]:
index_cluster = kmeans.predict(df.values)

In [27]:
index_each_cluster = []
for i in range(100):
    index_each_cluster.append([idx for idx in range(len(index_cluster)) if index_cluster[idx] == i])

In [28]:
cluster0 = index_each_cluster[0]
cluster0[:3]

[1040, 1086, 2529]

In [29]:
show_skill(1040)

['Top Skills: ', 'Finance', 'Banking', 'Retail Banking']
['Other Skills: ', 'Risk Management', 'Credit', 'Business Strategy', 'Corporate Finance', 'Change Management', 'Sales Management', 'Financial Analysis', 'Investment Banking', 'Trade Finance', 'Structured Finance', 'Management', 'Team Management', 'Banque', "Banque d'investissement", 'Gestion des risques', 'Banque de détail', 'Analyse financière', "Finance d'entreprise", 'Gestion de projet', "Gestion d'équipe", 'Stratégie commerciale', 'Crédit', 'Risque de crédit', 'Gestion de portefeuille']


In [30]:
show_skill(1086)

['Top Skills: ', 'Retail', 'Apparel', 'Fashion']
['Other Skills: ', 'Merchandise Planning', 'Merchandising', 'Inventory Management', 'Sports', 'Project Management', 'Operations Management', 'Event Planning', 'Project Planning', 'Excel', 'Management', 'Leadership', 'Trend']


In [31]:
show_skill(2529)

['Top Skills: ', 'Banking', 'Retail Banking', 'Commercial Management']
['Other Skills: ', 'Financial Analysis', 'Credit Analysis', 'Credit', 'Investment Banking', 'Corporate Finance', 'Portfolio Management', 'Insurance', 'Financial Modeling', 'Chemistry', 'SAP', 'Team management', 'Management', 'Accountability', 'Strategic Thinking', 'English', 'Italian', 'Arabic', 'Prospection', 'Management control', 'Analyse de crédit', 'Développement commercial', 'Nego', 'Integrity', 'Outlook Vision', 'Business Accumen']


# compare with the match function I wrote

In [32]:
match(df, 1040, 10)

[1040, 6965, 6978, 7507, 30511, 2529, 3028, 34361, 24812, 3528]

In [33]:
show_skill(1040)

['Top Skills: ', 'Finance', 'Banking', 'Retail Banking']
['Other Skills: ', 'Risk Management', 'Credit', 'Business Strategy', 'Corporate Finance', 'Change Management', 'Sales Management', 'Financial Analysis', 'Investment Banking', 'Trade Finance', 'Structured Finance', 'Management', 'Team Management', 'Banque', "Banque d'investissement", 'Gestion des risques', 'Banque de détail', 'Analyse financière', "Finance d'entreprise", 'Gestion de projet', "Gestion d'équipe", 'Stratégie commerciale', 'Crédit', 'Risque de crédit', 'Gestion de portefeuille']


In [34]:
show_skill(6965)

['Top Skills: ', 'Retail Banking', 'Finance', 'HR strategy']
['Other Skills: ', 'Corporate Banking', 'Change Management', 'Corporate Finance', 'CRM', 'Web design', 'Key Performance Indicators', 'Project Management', 'Event Planning', 'Recruiting', 'Workforce Planning', 'AML', 'Consulting', 'Learning management system', 'Microsoft PowerPoint', 'Microsoft Excel', 'Microsoft Word', 'WordPress', 'Leadership', 'Team Management', 'Public Speaking', 'Team Building', 'Teaching', 'Coaching', 'Mentoring', 'Management', 'Italian', 'Portuguese', 'Spanish', 'English', 'French', 'Business-to-Business (BtoB)', 'Audit Accounting', 'Social Networking', 'Transformation Lean', 'Digital Learning', 'Learning Engeener', 'Sourcing Engineering', 'Projet Microsoft', 'Financal Analisis', 'GPEC', 'Team Performance', 'Compliance', 'Gestion des risques', 'Ressources humaines', 'Développement organisationnel', 'Formation', 'Banque']


In [35]:
show_skill(6978)

['Top Skills: ', 'Retail Banking', 'Investment Banking', 'Management']
['Other Skills: ', 'Financial Services', 'Credit Risk', 'Risk Management', 'Financial Risk', 'Management Consulting', 'Operational Risk', 'Internal Audit', 'Financial Markets', 'Marketing digital', 'Private Banking', 'Structured Finance']


In [36]:
cluster2 = index_each_cluster[2]
cluster2[:3]

[245, 316, 561]

In [37]:
show_skill(245)

['Top Skills: ', 'Management', 'Gestion de projet', 'Microsoft Excel']
['Other Skills: ', 'Entrepreneuriat', 'Planification stratégique', 'Business Strategy', 'Marketing Strategy', 'Marketing', 'Microsoft Office', 'Microsoft Word', 'PowerPoint', 'Service client', 'Leadership', 'Formation']


In [38]:
show_skill(316)

['Top Skills: ', 'Microsoft Excel', 'Management', 'Leadership']
['Other Skills: ', 'Microsoft Word', 'PowerPoint', 'Microsoft Office', 'Gestion de projet', 'Planification stratégique', 'Service client', 'Vente']


In [39]:
show_skill(561)

['Top Skills: ', 'Microsoft Excel', 'Microsoft PowerPoint', 'Management']
['Other Skills: ', 'Marketing', 'Microsoft Office', 'Microsoft Word', 'Adobe Photoshop', 'Parler en public', 'Anglais', 'Travail d’équipe', 'Esprit d’équipe', 'Droit', 'Réseaux sociaux', 'Gestion de Projet', 'Analyse financière', 'Visual Basic pour Applications (VBA)', 'Espagnol']
